In [1]:
!pip install geocoder

     |████████████████████████████████| 98 kB 2.7 MB/s 


In [2]:
import json
import geocoder
import math
import csv
import numpy as np
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/"MyDrive"/"Colab Notebooks"/"DataMining"/"for_teacher"/

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/DataMining/for_teacher


In [4]:
!ls

b_lvr_land_a.csv.json  category.json


查經緯度

In [5]:
def get_geographic_coordinates(address):
    try:
      ret = geocoder.arcgis(address)
      return(ret.latlng)
    except:
      #print ERROR ADDRESS if address can't be parsed  
      print("ERROR")

查詢距離

In [6]:
def getDistance(latA, lonA, latB, lonB):
  try:
    #print(latA, lonA, latB, lonB)
    ra = 6378140  # 赤道半徑
    rb = 6356755  # 極半徑
    flatten = (ra - rb) / ra  # Partial rate of the earth
    # change angle to radians
    radLatA = math.radians(latA)
    radLonA = math.radians(lonA)
    radLatB = math.radians(latB)
    radLonB = math.radians(lonB)

    pA = math.atan(rb / ra * math.tan(radLatA))
    pB = math.atan(rb / ra * math.tan(radLatB))
    x = math.acos(math.sin(pA) * math.sin(pB) + math.cos(pA) * math.cos(pB) * math.cos(radLonA - radLonB))
    c1 = (math.sin(x) - x) * (math.sin(pA) + math.sin(pB)) ** 2 / math.cos(x / 2) ** 2
    c2 = (math.sin(x) + x) * (math.sin(pA) - math.sin(pB)) ** 2 / math.sin(x / 2) ** 2
    dr = flatten / 8 * (c1 - c2)
    distance = ra * (x + dr)
    distance = round(distance / 1000, 4)
    return distance
  except:
    return 0

查詢附近房地產資訊

In [21]:
def find_nearby(address,building_state,lat,lng,range):
  building_type={
      1:"住宅大樓(11層含以上有電梯)",2:"公寓(5樓含以下無電梯)",3:"華廈(10層含以下有電梯)",4:"透天厝"
  }
  nearby=[]
  avg1={
    "transaction_sign":{},
    "type":{},
    "building_state":{},
    "main_use":{},
    "building_materials":{},
    "berth_category":{},
    "residential_guard":{},
    "elevator":{}
  }
  avg2={
    "land_shifting_total_area":0,
    "building_shifting_total_area":0,
    "transaction_land_num":0,
    "transaction_building_num":0,
    "transaction_berth_num":0,
    "building_age":0,
    "building_room_num":0,
    "building_hall_num":0,
    "building_bathroom_num":0,
    "building_compartmented_num":0,
    "building_price":0,
    "unit_price":0,
    "berth_area":0,
    "berth_price":0,
  }
  with open("b_lvr_land_a.csv.json", newline='') as f:
    data = json.load(f)
    for estate in data:
      #房屋同類型
      if address==estate["address"]:
        nearby.append(estate)
      else:
        if building_state==5:
          if not(lat==estate["lat"] and lng==estate["lng"]):
            distance=getDistance(lat,lng,estate["lat"],estate["lng"])
            if distance<=range:
              #print(distance,estate)
              nearby.append(estate)
          
        elif estate["building_state"]==building_type[building_state]:
          if not(lat==estate["lat"] and lng==estate["lng"]):
            distance=getDistance(lat,lng,estate["lat"],estate["lng"])
            if distance<=range:
              #print(distance,estate)
              nearby.append(estate)
  #print(len(nearby))
  for estate in nearby:
    for title in avg1:
      if not (estate[title] in avg1[title]):
        avg1[title][estate[title]]=1
      else:
        avg1[title][estate[title]]+=1
    for title in avg2:
      
      avg2[title]+=float(estate[title])

  return nearby,avg1,avg2

In [8]:
def percentage(a,b):
  return round(float(a)/float(b)*100,2)

In [19]:
def nearby_infor(building_state,address,range):
  latlng=get_geographic_coordinates(address)
  nearby,avg1,avg2 = find_nearby(address,building_state,latlng[0],latlng[1],range)

  total=len(nearby)
  if(total>0):
    print("-"*50)
    print(f"查詢的地點周遭有{total}筆資料")
    print("-"*50)
    for i in avg1:
      print(f"{i}:")
      for j in avg1[i]:
        print(f"{j} {percentage(avg1[i][j],total)}%")
      print("-"*50)
    for i in avg2:
      print(f"{i}平均為{round(float(avg2[i])/total,3)}")
      print("-"*50)
    for i,data in enumerate(nearby):
      print(i,data["address"])
    return 
  else:
    return "ERROR"

周圍房屋資料分布

In [22]:
address=input("請輸入欲查詢之地址: ")
building_state=input("請輸入欲查詢之房屋類別 1.住宅大樓(11層含以上有電梯),2.公寓(5樓含以下無電梯),3.華廈(10層含以下有電梯),4.透天,5.全部: ")
range=input("請輸入查詢範圍(單位:KM): ")
nearby_infor(int(building_state),address,float(range))

請輸入欲查詢之地址: 台中市西區大忠南街
請輸入欲查詢之房屋類別 1.住宅大樓(11層含以上有電梯),2.公寓(5樓含以下無電梯),3.華廈(10層含以下有電梯),4.透天,5.全部: 1
請輸入查詢範圍(單位:KM): 1
--------------------------------------------------
查詢的地點周遭有18筆資料
--------------------------------------------------
transaction_sign:
房地(土地+建物)+車位 66.67%
房地(土地+建物) 33.33%
--------------------------------------------------
type:
都市：其他:第二種住宅區 77.78%
都市：其他:第四種商業區 5.56%
非都市： ; 非都市編定： 5.56%
都市：其他:第三種住宅區。 11.11%
--------------------------------------------------
building_state:
住宅大樓(11層含以上有電梯) 100.0%
--------------------------------------------------
main_use:
住家用 83.33%
集合住宅 16.67%
--------------------------------------------------
building_materials:
鋼筋混凝土造 94.44%
ＲＣ造 5.56%
--------------------------------------------------
berth_category:
坡道平面 50.0%
坡道機械 11.11%
0 33.33%
升降平面 5.56%
--------------------------------------------------
residential_guard:
1 100.0%
--------------------------------------------------
elevator:
1 100.0%
--------------------------------------------------


找出離群值

In [11]:
def find_different(text,building_state):
  too_low=[]
  too_high=[]
  with open("b_lvr_land_a.csv.json", newline='') as f:
    data = json.load(f)
    for estate in tqdm(data):
      nearby_data,avg1,avg2=find_nearby(estate["address"],int(building_state),estate["lat"],estate["lng"],1)
      if(len(nearby_data)>=10):
        price=[]
        for i in nearby_data:
          price.append(float(i[text]))
        #print(price)
        np_price = np.array(price)
        q1 = np.percentile(np_price, 25)
        q3 = np.percentile(np_price, 75)
        IQR = q3-q1
        avg=np.mean(np_price)
        #print(avg)
        #print(q1,q3,IQR,estate[22])
        if(float(estate[text])>q3+1.5*IQR):
          too_high.append(estate)
          print(estate["address"],estate[text],"高")
        elif(float(estate[text])<q1-1.5*IQR):
          too_low.append(estate)
          print(estate["address"],estate[text],"低")
  #產生離群值json檔
  # print(len(too_low))
  # with open("too_low1.json", 'w') as f:
  #   json.dump(too_low, f)
  # print(len(too_high))
  # with open("too_high1.json", 'w') as f:
  #   json.dump(too_high, f)


In [12]:
#以總價找離群值輸入:building_price
#以單坪價找離群值輸入:unit_price
building_state=input("請輸入欲查詢之房屋類別 1.住宅大樓(11層含以上有電梯),2.公寓(5樓含以下無電梯),3.華廈(10層含以下有電梯),4.透天,5.全部: ")
find_different("unit_price",building_state)

請輸入欲查詢之房屋類別 1.住宅大樓(11層含以上有電梯),2.公寓(5樓含以下無電梯),3.華廈(10層含以下有電梯),4.透天,5.全部: 1


  2%|▏         | 17/907 [00:00<00:17, 50.04it/s]

臺中市西區五權西三街１３３號１５樓 73631 高
臺中市東區東英路５８７號七樓 66455 高


  5%|▍         | 45/907 [00:00<00:17, 50.32it/s]

臺中市北區進化北路３９２之２號８樓之２ 35117 低
臺中市南屯區干城街２８０巷２弄１８號 242854 高


...

臺中市南屯區大昌街８６號 132594 高
臺中市南屯區大墩二街９８號五樓之２ 103563 高
臺中市南屯區大墩十街３３３號 115850 高


 99%|█████████▉| 900/907 [00:18<00:00, 50.54it/s]

臺中市東區德興一街１５７號 89913 高
臺中市東區十甲路４３１號９樓之８ 37291 低
臺中市西區存中街６８號１２樓之３ 108106 高


100%|██████████| 907/907 [00:18<00:00, 49.21it/s]

臺中市南區復興路３段１６５號 398117 高
臺中市南區復興路３段１６９號 414042 高
臺中市東區三賢街３０８號 111499 高
